# ECHAM6 executable

Checkout tagging branch and compile

    https://gitlab.awi.de/paleodyn/Models/echam6/-/tree/tagging

# Example yaml script

On ollie:

    /work/ollie/qigao001/scratch/test/share/pi_echam6_1m.yaml

## Explanation

echam::executable

    excutable compiled from the latest tagging branch

echam::add_namelist_changes::namelist.echam::wisoctl::

    nwiso: number of water tracers to use
    ntag/ntag1-5: only for development, set to 0

    ntags1_lat: number of water tracers for latitude, set to 3
    ntags2_lon: only for development, set to 0
    ntags3_sst/ntags4_rh2m/ntags5_wind10: number of water tracers for sea surface temperature, 2-metre relative humidity, and 10-metre wind speed, set to 3
    ntag_geo7: set to 7, water tracers for Antarctica, NH/SH land, ocean and sea ice
    ntags6_sinlon/ntags7_coslon: number of water tracers for sin and cos of longitude, set to 3
    ntag_sum: 25, =nwiso-3

    lupdate_tagmap: True (so tagmap will be updated at each timestep)
    tag_minsst: 268.15, minimum scaling factor for SST, similar for below
    tag_maxsst: 318.15
    tag_minrh2m: 0
    tag_maxrh2m: 1.6
    tag_minwind10: 0
    tag_maxwind10: 28.0

echam::forcing_sources::

    wiso_tagmap_1:: tagmap created for PI condition. Depending on land-sea mask





# Water tracing method

To trace each variable (e.g. SST), we need three water tracers:

    1) The first tracer traces evaporation from land and sea ice.
    2) The second tracer traces evaporation from open ocean, and the evaporation is scaled by the desired variable as E*(SST - tag_minsst)/(tag_maxsst - tag_minsst). Here E means evaporation.
    3) The third tracer is complementary to the second one over open ocean. It traces scaled evaporation from open ocean as E * [1 - (SST - tag_minsst)/(tag_maxsst - tag_minsst)]

To calculate precipitation-weighted open-oceanic source variables (e.g. SST)

    pre_weighted_var = var_scaled_pre / ocean_pre.values * (max_sf - min_sf) + min_sf

where var_scaled_pre is the precipitation from the second water tracer, ocean_pre is the the sum of water from the second and third tracers, max_sf and min_sf are the scaling factors. For latitude and sin/cos longitude, max_sf is 1 and min_sf is -1.

In the output, wiso variables (e.g. wisoaprt) will have 28 wisotypes. The first three is for water isotopes. The next three are three water tracers for latitude, and so on. You can also check through the tagmap.

Below is a python function to recover source properties.

In [ ]:
def source_properties(
    var_scaled_pre, ocean_pre, min_sf, max_sf, var_name,
    prefix = 'pre_weighted_', threshold = 2e-8,
    ):
    '''
    #---- Input
    var_scaled_pre : xarray.DataArray, precipitation scaled with var
    ocean_pre      : xarray.DataArray, open-oceanic precipitation
    min_sf         : parameter, minimum scaling factors
    max_sf         : parameter, maximum scaling factors
    var_name       : variable name
    
    #---- Output
    pre_weighted_var : precipitation-weighted open-oceanic source properties
    
    '''
    
    #---- Import packages
    import numpy as np
    
    #---- estimation on original time intervals
    pre_weighted_var = (var_scaled_pre / ocean_pre.values * (max_sf - min_sf) + min_sf).compute()
    pre_weighted_var.values[ocean_pre.values < threshold] = np.nan
    pre_weighted_var = pre_weighted_var.rename(prefix + var_name)
    
    if (var_name == 'sst'):
        pre_weighted_var.values[:] = pre_weighted_var.values[:] - 273.15
    
    if (var_name == 'rh2m'):
        pre_weighted_var.values[:] = pre_weighted_var.values[:] * 100
    
    return(pre_weighted_var)
